In [ ]:
import websocket
import threading
import json
import time
import csv
import os
from datetime import datetime

class ArbitrageMonitor:
    def __init__(self, symbol='sol', cost_threshold=0.005):
        self.symbol = symbol.lower()
        self.cost = cost_threshold
        self.state = {'spot': None, 'future': None}
        self.lock = threading.Lock()
        self.log_file = f"arbitrage_{self.symbol}.csv"
        self.init_log()
        self.prices_updated = {'spot': False, 'future': False}

    def init_log(self):
        if not os.path.exists(self.log_file):
            with open(self.log_file, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    'timestamp', 'symbol', 'spot_price', 'future_price',
                    'spread', 'profit', 'arbitrage_opportunity'
                ])

    def log(self, spot, future, spread, profit, is_opportunity):
        with open(self.log_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                datetime.utcnow().isoformat(), self.symbol.upper(),
                f"{spot:.4f}", f"{future:.4f}", f"{spread:.4f}",
                f"{profit:.4f}", 'YES' if is_opportunity else 'NO'
            ])

    def get_urls(self):
        return {
            "spot": f"wss://stream.binance.com:9443/ws/{self.symbol}usdt@trade",
            "future": f"wss://fstream.binance.com/ws/{self.symbol}usdt@trade"
        }

    def start(self):
        urls = self.get_urls()

        threading.Thread(target=self.run_ws, args=(urls['spot'], self.on_message_spot), daemon=True).start()
        threading.Thread(target=self.run_ws, args=(urls['future'], self.on_message_future), daemon=True).start()

        print(f"Monitoring arbitrage for {self.symbol.upper()}USDT... ")
        while True:
            time.sleep(1)

    def run_ws(self, url, on_message):
        ws = websocket.WebSocketApp(
            url,
            on_message=on_message,
            on_error=lambda ws, err: print("/!\ WebSocket error:", err),
            on_close=lambda ws, *args: print("/x\ WebSocket closed"),
            on_open=lambda ws: print(f"OK: Connected to {url}")
        )
        ws.run_forever()

    def on_message_spot(self, ws, message):
        data = json.loads(message)
        price = float(data['p'])

        with self.lock:
            self.state['spot'] = price
            self.prices_updated['spot'] = True
            self.check_arbitrage()

    def on_message_future(self, ws, message):
        data = json.loads(message)
        price = float(data['p'])

        with self.lock:
            self.state['future'] = price
            self.prices_updated['future'] = True
            self.check_arbitrage()

    def check_arbitrage(self):
        if not all(self.prices_updated.values()):
            return

        spot = self.state['spot']
        future = self.state['future']

        if spot and future:
            spread = future - spot
            profit = spread - self.cost
            is_opportunity = spread > self.cost

            if is_opportunity:
                print(f" {self.symbol.upper()} Arbitrage detected: Buy SPOT at ${spot:.4f}, Sell FUTURE at ${future:.4f} | Profit: ${profit:.4f}")
                self.log(spot, future, spread, profit, is_opportunity)
            elif int(time.time()) % 5 == 0:
                print(f"{self.symbol.upper()} Spot: ${spot:.4f} | Future: ${future:.4f} | Spread: ${spread:.4f}")

            # Reset the flags
            self.prices_updated = {'spot': False, 'future': False}

def monitor_multiple_symbols(symbols, cost_threshold=0.005):
    threads = []
    for symbol in symbols:
        monitor = ArbitrageMonitor(symbol=symbol, cost_threshold=cost_threshold)
        t = threading.Thread(target=monitor.start, daemon=True)
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

if __name__ == "__main__":
    symbols = ['eth', 'btc', 'sol', 'ada']
    monitor_multiple_symbols(symbols, cost_threshold=0.005)


Monitoring arbitrage for ETHUSDT... 
Monitoring arbitrage for BTCUSDT... 
Monitoring arbitrage for SOLUSDT... 
Monitoring arbitrage for ADAUSDT... 
OK: Connected to wss://fstream.binance.com/ws/solusdt@trade
OK: Connected to wss://stream.binance.com:9443/ws/solusdt@trade
OK: Connected to wss://fstream.binance.com/ws/ethusdt@trade
OK: Connected to wss://stream.binance.com:9443/ws/adausdt@trade
OK: Connected to wss://fstream.binance.com/ws/adausdt@trade
OK: Connected to wss://stream.binance.com:9443/ws/ethusdt@trade
OK: Connected to wss://fstream.binance.com/ws/btcusdt@trade
OK: Connected to wss://stream.binance.com:9443/ws/btcusdt@trade
BTC Spot: $103594.0300 | Future: $103538.4000 | Spread: $-55.6300
ETH Spot: $2358.3000 | Future: $2357.4700 | Spread: $-0.8300
BTC Spot: $103594.0300 | Future: $103538.4000 | Spread: $-55.6300
SOL Spot: $171.4100 | Future: $171.3500 | Spread: $-0.0600
ETH Spot: $2358.3000 | Future: $2357.4700 | Spread: $-0.8300
BTC Spot: $103594.0400 | Future: $103538.40